# Data preparation

### Additional requirements

In addition to the BAVARIA requirements, this notebook requires the python package [`pybedtools`](https://daler.github.io/pybedtools/)
and the [bedtools](https://bedtools.readthedocs.io/en/latest/) program to be installed.

### Overview

In this notebook, we download and prepare two PBMC single-cell ATAC seq datasets from 10X Genomics.
The two samples represent different versions of the 10X kit.
We will apply BAVARIA on the merged dataset in the 01_pbmc_integration notebook to demonstrate the effect of batch correction.

The result of this notebook is a h5ad file containing the dataset that is stored in `data/pbmc_10X.h5ad`.
An already prepared version of the dataset (`data/pbmc_10X.h5ad`) is also available at Google drive:
   https://drive.google.com/file/d/138_35KrnUB9Xjs4yDyzMQRqrkoqu_uwV/view?usp=sharing 

In [1]:
import os
from scipy.io import mmread
import pandas as pd
import anndata
import scanpy as sc
import numpy as np
from pybedtools import BedTool
from scipy.sparse import coo_matrix

In [2]:
!mkdir -p data

# 2 PBMC datasets
!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz -O data/atac_v1_pbmc_10k_fragments.tsv.gz
!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_10k/atac_v1_pbmc_10k_peaks.bed -O data/atac_v1_pbmc_10k_peaks.bed
!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_10k/atac_v1_pbmc_10k_singlecell.csv -O data/atac_v1_pbmc_10k_singlecell.csv

!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_pbmc_10k_nextgem/atac_pbmc_10k_nextgem_fragments.tsv.gz -O data/atac_pbmc_10k_nextgem_fragments.tsv.gz
!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_pbmc_10k_nextgem/atac_pbmc_10k_nextgem_peaks.bed -O data/atac_pbmc_10k_nextgem_peaks.bed
!wget https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_pbmc_10k_nextgem/atac_pbmc_10k_nextgem_singlecell.csv -O data/atac_pbmc_10k_nextgem_singlecell.csv


--2021-05-21 21:54:01--  https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_10k/atac_v1_pbmc_10k_fragments.tsv.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1955495674 (1,8G) [text/tab-separated-values]
Saving to: ‘data/atac_v1_pbmc_10k_fragments.tsv.gz’

100%[====================================>] 1.955.495.674 17,7MB/s   in 2m 5s  

2021-05-21 21:56:07 (14,9 MB/s) - ‘data/atac_v1_pbmc_10k_fragments.tsv.gz’ saved [1955495674/1955495674]

--2021-05-21 21:56:07--  https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_v1_pbmc_10k/atac_v1_pbmc_10k_peaks.bed
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting 

For this tutorial we will proceed with the peaks and filtered cells from the CellRanger pipeline, which we downloaded above.

First, we compile a master peak set by combining peaks from both datasets:

In [3]:
#make a master peak set
!cat data/atac_v1_pbmc_10k_peaks.bed data/atac_pbmc_10k_nextgem_peaks.bed | bedtools sort | bedtools merge > data/masterpeaks.bed


Next, we load the master peaks as BedTool object.

In [4]:
peak =pd.read_csv('data/masterpeaks.bed', sep='\t', header=None)
peak.columns=['chrom','start','end']

peak.loc[:, "ridx"] = range(peak.shape[0])
# remove sex chroms and chrom M
dfpeak = peak[~peak.chrom.isin(['chrX', 'chrY','chrM'])].copy()
dfpeak.loc[:,'idx'] = dfpeak.apply(lambda row: f'{row.chrom}:{row.start}-{row.end}', axis=1)
dfpeak.set_index('idx', inplace=True)
peak = BedTool.from_dataframe(dfpeak)


In [5]:
batches =  {
            'pbmc_10k': {'frag': 'data/atac_v1_pbmc_10k_fragments.tsv.gz',
                         'cells': 'data/atac_v1_pbmc_10k_singlecell.csv'},
            'pbmc_10k_nextgem': {'frag': 'data/atac_pbmc_10k_nextgem_fragments.tsv.gz',
                         'cells': 'data/atac_pbmc_10k_nextgem_singlecell.csv'},
           }

In [6]:
def get_fragment_bedtool(fragments, barcodes):
    """ Load and filter fragments
    
    Only valid cells (defined by is__cell_barcode) are used.
    """
    df = pd.read_csv(fragments,sep='\t', header=None)
    df.columns = ['chr','start','end','barcode', 'count']
    #bcf = pd.read_csv(keepbarcodes)
    barcodes = barcodes[barcodes.is__cell_barcode==1]
    barcodes.loc[:,'idx'] = range(barcodes.shape[0])
    df = pd.merge(df, barcodes, on='barcode', how='inner')[['chr','start','end','barcode', 'idx']]
    return BedTool.from_dataframe(df)



In [7]:
adatas = []
for batchname in batches:
    barcodes = pd.read_csv(batches[batchname]['cells'])
    barcodes = barcodes[barcodes.is__cell_barcode==1]
    barcodes.loc[:,"batch"] = batchname
    barcodes.set_index('barcode', inplace=True)
    
    frags = get_fragment_bedtool(batches[batchname]['frag'], barcodes)
    
    peakcounts = peak.intersect(frags,
                             wa=True,
                             wb=True).to_dataframe()
    sparse_data = np.asarray([np.ones(peakcounts.shape[0]),
                             peakcounts.name, peakcounts.itemRgb]).T
    sparse_data = np.unique(sparse_data, axis=0)
    mat = coo_matrix((sparse_data[:,0], (sparse_data[:,1], sparse_data[:,2])),
                     shape=(len(peak), len(barcodes)))
    adata = anndata.AnnData(mat.T.tocsr(), obs=barcodes, var=dfpeak)
    adatas.append(adata)


In [8]:
adata = anndata.concat(adatas, axis=0)
adata.obs_names_make_unique()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [9]:
# remove regions waith < %1 coverage across cells
regioncover = np.asarray(adata.X.sum(0)).flatten()
adata = adata[:, regioncover>=0.01*adata.shape[0]].copy()

The preprocessed and merged dataset is saved in `data/pbmc_10X.h5ad`

In [10]:
adata.write('data/pbmc_10X.h5ad')

... storing 'cell_id' as categorical
... storing 'batch' as categorical


In [11]:
adata

AnnData object with n_obs × n_vars = 18301 × 73228
    obs: 'total', 'duplicate', 'chimeric', 'unmapped', 'lowmapq', 'mitochondrial', 'passed_filters', 'cell_id', 'is__cell_barcode', 'TSS_fragments', 'DNase_sensitive_region_fragments', 'enhancer_region_fragments', 'promoter_region_fragments', 'on_target_fragments', 'blacklist_region_fragments', 'peak_region_fragments', 'peak_region_cutsites', 'batch'

In [12]:
adata.obs.batch.value_counts()

pbmc_10k_nextgem    9668
pbmc_10k            8633
Name: batch, dtype: int64

After the preprocessing, we obtain a 18301 cells by 73228 peaks count matrix. The two batches (pbmc_10k_next_gem and pbmc_10x) consist of 9668 and 8633 cells, respectively. We use this count matrix to fit the variational auto-encoder in the next tutorial notebook.